In [8]:
import csv
import time
import sys
import pandas as pd
import geopandas as gpd
import shapely
from shapely.ops import nearest_points
from scipy import spatial
from pyproj import Proj

In [2]:
 prefix = 'parallel'
 filename = 'xaa'
 colnames = ['record id', 'case no', 'date', 'block', 'iucr','primary type', 'description',
             'location description', 'arrest', 'domestic', 'beat', 'district', 'ward', 'community area',
             'fbi code', 'x coordinate', 'y coordinate', 'year', 'updated on', 'latitude',
             'longitude', 'location']
 crimesdf = pd.read_csv("{0}/{1}".format(prefix, filename), names=colnames, header=None)
 crimesdf.dropna(inplace=True)
 print(crimesdf.shape)
 del crimesdf['block']
 del crimesdf['iucr']
 del crimesdf['domestic']
 del crimesdf['beat']
 del crimesdf['district']
 del crimesdf['fbi code']
 del crimesdf['x coordinate']
 del crimesdf['y coordinate']
 del crimesdf['year']
 del crimesdf['updated on']
 del crimesdf['location']
 
 crimesdf['geometry'] = list(zip(crimesdf['latitude'], crimesdf['longitude']))
 crimesdf['geometry'] = crimesdf['geometry'].apply(shapely.geometry.Point)
 crimesgpd = gpd.GeoDataFrame(crimesdf)
 print(crimesgpd.iloc[0])

(1147, 22)
record id                                         8265244
case no                                          HT498458
date                               09/15/2011 08:00:00 AM
primary type                                     BURGLARY
description                                UNLAWFUL ENTRY
location description                            RESIDENCE
arrest                                               True
ward                                                   38
community area                                         15
latitude                                           41.954
longitude                                        -87.7719
geometry                POINT (41.95396528 -87.771918163)
Name: 0, dtype: object


In [4]:
# Read in the school locations data, fix the type of the UNIT_ID column, create shapely points from X/Y
# and create the geopandas DataFrame
schools = pd.read_csv("school-locations-2010-2011.csv", index_col=2)
schools['UNIT_ID'] = schools.index
schools = schools.astype({'UNIT_ID': int})
schools['geometry'] = list(zip(schools['X'], schools['Y']))
schools['geometry'] = schools['geometry'].apply(shapely.geometry.Point)
schools = gpd.GeoDataFrame(schools)
schools.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 688 entries, 4610 to 4210
Data columns (total 10 columns):
geometry     688 non-null object
SCHOOL_NM    688 non-null object
SCHOOL_ID    688 non-null int64
SCH_ADDR     688 non-null object
GRADE_CAT    688 non-null object
GRADES       688 non-null object
SCH_TYPE     688 non-null object
X            688 non-null float64
Y            688 non-null float64
UNIT_ID      688 non-null int64
dtypes: float64(2), int64(2), object(6)
memory usage: 59.1+ KB


In [6]:
 start_time2 = time.time()
 D = spatial.distance_matrix(crimesgpd['geometry'], schools['geometry'])
 nn = np.array([[np.min(d[i,]), np.argmin([i,])] for i in range(crimesgpd['geometry'].shape[0])])
 crimesgpd['scipy distance'] = nn[:,0]
 crimesgpd['scipy distance2'] = nn[:,1]
 end_time = time.time()
 print("That took {0} seconds".format(end_time - start_time))
 print(crimesgpd.head())

ValueError: not enough values to unpack (expected 2, got 1)

In [7]:
len(crimesgpd)

1147